In [1]:
from datasets import load_from_disk
from collections import defaultdict, deque
from tqdm import tqdm
import random

/home/connor/miniconda3/envs/lmsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chrono_data = load_from_disk('chrono_data/chrono_train_data/')

In [5]:
curr_batch = 0
queue = defaultdict(deque)
batch = defaultdict(deque)
batch_size = 512
num_data = len(chrono_data) 
model_cnts = defaultdict(int)
gamma = 0.9999
epsilon = 0.01
random.seed(42)

queue[0] = deque([i for i in range(num_data)])

pbar = tqdm(total=num_data)

num_proc = 0
while queue[curr_batch]:
    curr = queue[curr_batch].popleft()
    if random.random() < min(1-epsilon, 1-gamma**(model_cnts[chrono_data[curr]['model_a']] + model_cnts[chrono_data[curr]['model_b']])):
        queue[curr_batch+1].append(curr)
    else:
        num_proc += 1
        batch[curr_batch].append(curr)
        pbar.update(1)
        model_cnts[chrono_data[curr]['model_a']] += 1
        model_cnts[chrono_data[curr]['model_b']] += 1
        if len(batch[curr_batch]) == batch_size:
            queue[curr_batch+1].extend(queue[curr_batch])
            queue[curr_batch].clear()
            curr_batch += 1


pbar.close()

  0%|          | 0/2247925 [00:00<?, ?it/s]

 48%|████▊     | 1073798/2247925 [4:02:31<7:59:51, 40.78it/s] 

KeyboardInterrupt: 

##Processing Naive Replay Buffer Data

In [3]:
#selects random point from the past with probability epsilon, with replacement
from collections import deque
batch_size = 512
data_order = [i for i in range(512 * 5)]
replay_size = 52
new_size = 460
num_data = len(chrono_data)
replay = [i for i in range(512 * 5)]

data_flow = deque([i for i in range(512*5, num_data)])
while data_flow:
    new_elements = []
    replay_elements = []
    while data_flow and len(new_elements) < new_size:
        new_elements.append(data_flow.popleft())
    for _ in range(50):
        random_index = random.randrange(len(replay))
        value = replay[random_index]
        replay_elements.append(replay.pop(random_index))
        
    replay += new_elements
    data_order += new_elements + replay_elements


In [4]:
replay_buffer = [chrono_data[i] for i in data_order]

In [8]:
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(replay_buffer)
dataset = Dataset.from_pandas(df)

In [10]:
dataset.save_to_disk("chrono_data/naive_replay_buffer_eps_0.1")

Saving the dataset (6/6 shards): 100%|██████████| 2492025/2492025 [00:05<00:00, 476652.08 examples/s]
